In [1]:
#Import libraries
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup 

#Wikipedia's URL, connect to the page and get data
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 

#Read content
soup = BeautifulSoup(r.content, 'html5lib') 

#Find table with information
table = soup.find('table', attrs = {'class':'wikitable sortable'}) 

#Create an empty list to put data
my_list = []
#Go through table rows and extract text from columns and add into my_list
for idx, row in enumerate(table.findAll('td')):
    x  = row.text
    x = x.replace('\n','')
    my_list.append(x)

#Create chunks fuction to split list into different lists
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

#Headers of data frame
header = ['Postcode','Borough', 'Neighbourhood']
#Data of dataframe
data = list(chunks(my_list,3))

#Finally, create my dataframe with table data
df = pd.DataFrame(data,columns=header)

#Remove Not assigned cells
df = df[df['Borough'] != 'Not assigned']
#Group cells by Postcode and Borough and group neighbourhoods using ', ' as delimiter
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x.dropna().unique())).reset_index()
#Find not assigned neighourhoods and if that's the case use the Borough name, otherwise keep the neighbourhood value
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'],df['Neighbourhood'])

#Shape
df.shape

(103, 3)